In [1]:
import pickle as pkl
import pandas as pd
import numpy as np

In [2]:
with open('./output.pkl','rb') as f:
    datas = pkl.load(f)

In [3]:
datas = [x['AUS_model'].iloc[:,[0,-1]] for x in datas]

In [4]:
true = [data.iloc[:,-1] for data in datas]

In [5]:
true = pd.concat(true, axis=1).iloc[:,:-1]

In [6]:
true

,CYEAR=2000,CYEAR=2001,CYEAR=2002,CYEAR=2003,CYEAR=2004,CYEAR=2005,CYEAR=2006,CYEAR=2007,CYEAR=2008,CYEAR=2009,CYEAR=2010,CYEAR=2011,CYEAR=2012
0,4.475183,4.380841,3.003171,2.770735,2.343612,2.668733,3.538487,2.332362,4.352643,1.820112,2.845226,3.303850,1.762780
1,5.030644,2.035681,3.403482,3.391598,3.648011,3.422373,2.425093,3.067561,4.182300,1.044141,6.206931,1.462691,4.820785
2,6.300000,6.700000,6.400000,5.900000,5.400000,5.000000,4.800000,4.400000,4.200000,5.600000,5.200000,5.100000,5.200000


In [7]:
pred = [data.iloc[:,0] for data in datas]

In [8]:
pred = pd.concat(pred, axis=1).iloc[:,:-2]

In [9]:
pred

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,5.178518,6.289144,5.053928,4.238345,4.172746,4.098124,3.951116,3.337889,3.953211,2.159922,2.567804,4.051530
1,4.731508,2.492307,3.999616,3.833587,3.465822,2.977892,1.970480,3.127066,3.143707,2.104447,4.458294,2.567048
2,6.653108,5.644829,6.394327,5.667459,5.115726,5.059286,4.932859,4.838956,5.211691,4.678940,6.255990,4.553951


In [10]:
rw_pred_std = np.std(true.values[:,1:] - true.values[:,:-1], axis=1)

In [11]:
rw_pred_std

array([1.22401319, 2.64403164, 0.51552292])

In [12]:
pred_std = np.std(true.values[:,1:] - pred.values, axis=1)

In [13]:
pred_std

array([1.32578127, 1.9666636 , 0.55790062])